In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
# defining transforms for mnist
my_transforms = transforms.Compose([
    transforms.ToTensor()
])

importing the dataset from torchvision.

In [ ]:
train_dataset = datasets.FashionMNIST(root="./data", download=True, transform=my_transforms, train=True)
test_dataset = datasets.FashionMNIST(root="./data", download=True, transform=my_transforms, train=False)

creating dataloader out of the dataset.

In [ ]:
batch_size=128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# here we are looking at test_dataloader as more of a validation set
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

now lets look at our dataset.

In [ ]:
import matplotlib.pyplot as plt

for X, y in train_dataloader:
    print("Shape of X: ", X.shape)
    print("Length of y: ", len(y))
    break

In [ ]:
plt.imshow(X[0].permute(1,2,0))
print(y[0])

In [ ]:
def look_in_net(net, X):
    out = X
    for layer in net:
        out = layer(out)
        print(f"For layer : {layer.__class__.__name__} shape : {out.shape}")

Now lets create our model.

In [ ]:
import torch.nn.functional as F

random_cnn = nn.Sequential(nn.Conv2d(1, 96, kernel_size=3,stride=1, padding=1),
                          nn.ReLU(),
                          nn.Conv2d(96, 128, kernel_size=5, padding=2),
                          nn.ReLU(),
                          nn.Flatten(),
                          nn.Linear(128 * 7 * 7, 240), 
                          nn.Dropout(p=0.5),
                          nn.Linear(240, 10))

#from https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/
random_cnn = nn.Sequential( nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(4),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                           
                            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(4),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                           
                            nn.Flatten(),
                            nn.Linear(4 * 7 * 7, 40), 
                            nn.Dropout(p=0.5),
                            nn.Linear(40, 10)
                           )

#from https://www.kaggle.com/sdelecourt/cnn-with-pytorch-for-mnist
random_cnn = nn.Sequential(
                  nn.Conv2d(1, 32, kernel_size=5),
                  nn.ReLU(),
                  nn.Conv2d(32, 32, kernel_size=5),
                  nn.MaxPool2d(2),
                  nn.ReLU(),
                  nn.Dropout(p=0.5),
                  nn.Conv2d(32,64, kernel_size=5),
                  nn.MaxPool2d(2),
                  nn.ReLU(),
                  nn.Dropout(p=0.5),
                  nn.Flatten(),
                  nn.Linear(3*3*64, 256),
                  nn.Dropout(p=0.5),
                  nn.Linear(256, 10)
)

In [ ]:
X.shape

In [ ]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:


net = CNN()

net(X)

In [ ]:
look_in_net(random_cnn, X)

Now using fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
optimizer = torch.optim.Adam(random_cnn.parameters(), lr=0.001)

In [ ]:
data = DataLoaders(train_dataloader, test_dataloader)
learner = Learner(data, CNN(), loss_func=F.nll_loss, opt_func=Adam, metrics=accuracy)

# cannot use cross entropy loss, booleanvalue error

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(1,2e-2)

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(10,2e-4)